# Heat equation

c.f. https://teamcoil.sp.u-tokai.ac.jp/lectures/EL1/Poisson/index.html  
https://www.research.kobe-u.ac.jp/csi-viz/members/kageyama/lectures/H22_FY2010_latter/2nd_Sim_School/index.ja.html

$$
\begin{align}
\frac{\partial \phi}{\partial t} = c \Delta \phi
\end{align}
$$

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.animation as animation
# https://qiita.com/fhiyo/items/0ea94b0de6cd5c76d67a
from IPython.display import HTML
import pickle
import numba

In [ ]:
N = 100
X = 1.0
T = 100
thermal_conductivity = 0.1
time_step = 5
center = np.array((N // 2, N // 2))
delta = X / N
rho = np.zeros((N, N))

In [ ]:
# initial value
for i in range(N):
    for j in range(N):
        if np.linalg.norm(center - (i, j))*delta < 0.05:
            rho[i, j] = 10

In [ ]:
@numba.jit
def calc_variation_at(i, j, phi: np.ndarray):
    # discrete Laplacian * thermal_conductivity
    return (phi[i+1, j]+phi[i-1, j]+phi[i, j+1]+phi[i, j-1]-4*phi[i, j]) * thermal_conductivity  # * delta_t / (delta**2)

@numba.jit
def calt_phi(prev_phi):
    phi = np.zeros((N, N), dtype=numba.float32)
    for i in range(1, N-1):
        for j in range(1, N-1):
            phi[i, j] = prev_phi[i, j] + calc_variation_at(i, j, prev_phi)

    return phi

In [ ]:
%%time

solutions = [rho]
phi = rho

for t in range(1, T):
    for _ in range(time_step):
        phi = calt_phi(phi)
    solutions.append(phi)

## Visualization of the solution of heat equation

In [ ]:
phi = solutions[0]
vmin = vmin=np.min(phi)
print(np.min(phi), np.max(phi))
vmax=np.max(phi)

fig, ax = plt.subplots()
xs, ys = np.meshgrid(np.arange(N), np.arange(N))
zs = phi[xs, ys]
xs_, ys_ = np.meshgrid(np.arange(N)*delta, np.arange(N)*delta)
im = ax.pcolormesh(xs_, ys_, zs, vmin=vmin, vmax=vmax, cmap='viridis') # or jet
ax.contour(xs_, ys_, zs, linewidths=1, alpha=0.5)
fig.colorbar(im, ax=ax)
ax.set_aspect('equal')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

In [ ]:
phi = solutions[5]
print(np.min(phi), np.max(phi))
fig, ax = plt.subplots()
xs, ys = np.meshgrid(np.arange(N), np.arange(N))
zs = phi[xs, ys]
xs_, ys_ = np.meshgrid(np.arange(N)*delta, np.arange(N)*delta)
im = ax.pcolormesh(xs_, ys_, zs, vmin=vmin, vmax=vmax, cmap='viridis') # or jet
ax.contour(xs_, ys_, zs, linewidths=1, alpha=0.5)
fig.colorbar(im, ax=ax)
ax.set_aspect('equal')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

## 2D animation

In [ ]:
ims = []

fig = plt.figure()

for sol in solutions:
    xs, ys = np.meshgrid(np.arange(N), np.arange(N))
    zs = sol[xs, ys]
    xs_, ys_ = np.meshgrid(np.arange(N)*delta, np.arange(N)*delta)
    im = plt.imshow(zs, vmin=vmin, vmax=vmax, cmap='viridis')
    ims.append([im])

In [ ]:
ani = animation.ArtistAnimation(fig, ims, interval=100)
ani.save("heat_2d.gif", writer="imagemagick")
HTML(ani.to_jshtml())

## 3D animation

In [ ]:
%%time

ims = []

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

for sol in solutions:
    xs, ys = np.meshgrid(np.arange(N), np.arange(N))
    zs = sol[xs, ys]
    xs_, ys_ = np.meshgrid(np.arange(N)*delta, np.arange(N)*delta)
    im = ax.plot_surface(xs_, ys_, zs, vmin=vmin, cmap='viridis')
    plt.xlabel('x')
    plt.ylabel('y')
    ims.append([im])

In [ ]:
ani = animation.ArtistAnimation(fig, ims, interval=100)
ani.save("heat_3d.gif", writer="imagemagick")
HTML(ani.to_jshtml())